# Introduction

In this notebook, we will experiment with how user queries are handled in our veterinary information retrieval system. Several collections have already been set up in the Chroma database, allowing us to directly perform information retrieval without additional setup. This environment enables us to test and refine the process of transforming user input into actionable queries and retrieving relevant information from our knowledge base.

In [1]:
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from langchain_chroma import Chroma

persist_directory = '../chroma/textbook_test_Nutrition'
id_key = "doc_id"

open_clip_embeddings = OpenCLIPEmbeddings(model_name="ViT-g-14", checkpoint="laion2b_s34b_b88k")

# Vectorstore for summaries (for similarity search)
vectorstore = Chroma(
    collection_name="summaries",
    persist_directory=persist_directory,
    embedding_function=open_clip_embeddings
)
# Persistent docstore for originals (all modalities)
docstore = Chroma(
    collection_name="originals",
    persist_directory=persist_directory,
    embedding_function=open_clip_embeddings
)

# Instantiate the retriever

class UnifiedRetriever:
        
        def __init__(self, vectorstore, docstore, id_key="doc_id"):
            self.vectorstore = vectorstore
            self.docstore = docstore
            self.id_key = id_key
            self._collection = docstore._collection

        def retrieve(self, query, k=5):
            results = self.vectorstore.similarity_search_with_score(query, k=k)
            output = []
            for doc, score in results:
                doc_id = doc.metadata.get(self.id_key)
                try:
                    original = self._collection.get(ids=[doc_id], include=["documents", "metadatas"])
                    original_doc = original["documents"][0] if original["documents"] else None
                    original_meta = original["metadatas"][0] if original["metadatas"] else None
                except Exception as e:
                    original_doc = None
                    original_meta = None
                output.append({
                    "summary": doc.page_content,
                    "original": original_doc,
                    "original_metadata": original_meta,
                    "summary_metadata": doc.metadata,
                    "score": score
                })
            return output

retriever = UnifiedRetriever(vectorstore, docstore, id_key=id_key)

# Handling User Input: Analysis Image

Let's use a image of a under weighted cat. This cat is considerablly skinny with bones showing. 

  ![Skinney Cat](./skinny_cat.jpg)

In [3]:
query = "What's going on with my cat? What should I do?" 
image_path = "./skinny_cat.jpg"

import base64
import ollama
import os

# --- Configuration for the image ---
# IMPORTANT: Adjust this path if your cat.jpg is in a different location
# image_model = "minicpm-v:8b" # Or "llava:7b" or another suitable vision model you have installed via Ollama
# image_model = "llava:7b" # 

#This instruct version, q8_0 weight format fits MacBook M1 Pro better
# image_model = "llama3.2-vision:11b-instruct-q8_0" # 
image_model = "gemma3:4b-it-q8_0" # 


# --- 1. Generate a textual summary of the image using an LLM ---
print(f" ⏳ Processing image: {image_path}")

image_summary = "Could not generate image summary." # Default in case of error
if not os.path.exists(image_path):
    print(f"Error: Image file not found at {image_path}. Please check the path.")
else:
    try:
        # Read and encode image in base64
        with open(image_path, 'rb') as f:
            image_data = base64.b64encode(f.read()).decode('utf-8')

        # Updated prompt for detailed image summarization
        image_summarization_prompt = """From a feline veterinary stand point, provide a highly detailed and objective 
                description of the image. Focus on all observable elements, actions, 
                objects, subjects, their attributes (e.g., color, size, texture), 
                their spatial relationships, and any discernible context or implied scene. 
                Also focus on all possible health issue.
                Describe any text present in the image. This description must be exhaustive 
                and purely factual, capturing every significant visual detail to serve as a 
                comprehensive textual representation for further analysis by another AI model. 
                If the image is entirely irrelevant or contains no discernible subject, 
                state "No relevant visual information."."""

        # Send image to ollama for vision model processing
        response = ollama.chat(
            model=image_model,
            messages=[
                {
                    'role': 'user',
                    'content': image_summarization_prompt,
                    'images': [image_data]
                }
            ]
        )
        image_summary = response['message']['content']
        print("--- Generated Image Summary ---")
        print(image_summary)

    except Exception as e:
        print(f"Error processing image with Ollama: {e}")

# This 'image_summary' can now be used along with your user's text query
# for retrieval or further processing in your RAG pipeline.

 ⏳ Processing image: ./skinny_cat.jpg
--- Generated Image Summary ---
Okay, here’s a highly detailed and objective description of the image from a feline veterinary standpoint, focusing on all observable details and potential health concerns.

**Overall Assessment:**

The image depicts a single domestic cat (likely a mixed breed, indeterminate age) exhibiting signs of significant emaciation and distress. The animal's overall condition strongly suggests malnutrition, possible dehydration, and potential underlying health issues warranting immediate veterinary attention.

**Subject – The Cat:**

*   **Posture & Movement:** The cat is in a hunched posture, with its head lowered and its body partially collapsed. It is actively engaged in consuming food from a bowl. This posture may indicate pain or discomfort, potentially related to skeletal or muscular issues.
*   **Coat:** The coat appears thin, matted, and patchy. The fur is sparse, with noticeable areas of missing or damaged hair. The t

# Handling User Input: Refine Query

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Define the LLM for query refinement (using the same model as your RAG chain if appropriate)
# Compressed, Distilled Qwen, Response often in CoT <think></think>
# query_refinement_model = ChatOllama(model="deepseek-r1:7b-qwen-distill-q8_0")

#compressed that fits m1 pro, use less RAM. No CoT
query_refinement_model = ChatOllama(model="llama3.2:3b-text-q8_0")

# Prompt for query refinement
query_refinement_prompt = ChatPromptTemplate.from_template(
    """You are an intelligent assistant. Your task is to rephrase and expand the given user query \
into a more detailed and context-rich query that can be used to retrieve relevant information \
from a veterinary knowledge base. Use the provided image description to add visual context \
and relevant keywords to the refined query. Focus on adding relevant keywords, clarifying intent, \
and anticipating related information that might be helpful. The output should be a single, refined query.

Original query: {original_query}
Image description: {image_summary}"""
)

# Create the query refinement chain
query_refinement_chain = (
    {
        "original_query": RunnablePassthrough(),
        "image_summary": RunnablePassthrough()
    }
    | query_refinement_prompt
    | query_refinement_model
    | StrOutputParser()
)# --- Demonstration of query refinement and then retrieval with scores ---


print(f"Original user query: {query}")
print(f"Image Summary: {image_summary}")

refined_query = query_refinement_chain.invoke(
    {"original_query": query, "image_summary": image_summary}
)

print("-"*80)
print(f"Refined query: {refined_query}")


Original user query: What's going on with my cat? What should I do?
Image Summary: Okay, here’s a highly detailed and objective description of the image from a feline veterinary standpoint, focusing on all observable details and potential health concerns.

**Overall Assessment:**

The image depicts a single domestic cat (likely a mixed breed, indeterminate age) exhibiting signs of significant emaciation and distress. The animal's overall condition strongly suggests malnutrition, possible dehydration, and potential underlying health issues warranting immediate veterinary attention.

**Subject – The Cat:**

*   **Posture & Movement:** The cat is in a hunched posture, with its head lowered and its body partially collapsed. It is actively engaged in consuming food from a bowl. This posture may indicate pain or discomfort, potentially related to skeletal or muscular issues.
*   **Coat:** The coat appears thin, matted, and patchy. The fur is sparse, with noticeable areas of missing or damage

# Query Decomposition

In [8]:
from langchain_core.output_parsers import JsonOutputParser
# Prompt for query decomposition
query_decomposition_prompt = ChatPromptTemplate.from_template(
    """You are an intelligent assistant. Your task is to break down the given complex query \
into a list of simpler, focused sub-queries. Each sub-query should be a standalone question \
that can be used to retrieve specific information from a veterinary knowledge base. \
Present the sub-queries as a JSON array of strings, where each string is a sub-query. \

Complex query: {refined_query}"""
)

# Create the query decomposition chain
query_decomposition_chain = (
    query_decomposition_prompt  
    | query_refinement_model    
    | JsonOutputParser() 
)

# --- Demonstration of query decomposition ---

print(f"Original refined query: {refined_query[:300]} ....")

decomposed_queries = query_decomposition_chain.invoke({"refined_query": refined_query})

print("-" * 80)
# print(f"Decomposed queries:\n{decomposed_queries}")

print(f"There are {len(decomposed_queries)} queries after decomposition \n")
print(f"Here's a example of the first one: {decomposed_queries[0]}")


Original refined query: <think>
Okay, so I need to help rephrase and expand the user's query into something more detailed and context-rich. The goal is to make it easier for a veterinary knowledge base to retrieve relevant information. They also provided an image summary with a lot of specific details about a cat in distre ....
--------------------------------------------------------------------------------
There are 14 queries after decomposition 

Here's a example of the first one: What is my cat's overall health status?


# Contextual Retrievals 

In [ ]:
# Assume decomposed_queries is a list of query strings
# and retriever is already instantiated

seen_doc_ids = set()
all_results = []

for query in decomposed_queries:
    results = retriever.retrieve(query, k=5)
    unique_results = []
    for res in results:
        doc_id = res.get('doc_id') or res.get('summary_metadata', {}).get('doc_id')
        if doc_id and doc_id not in seen_doc_ids:
            seen_doc_ids.add(doc_id)
            unique_results.append(res)
    if unique_results:
        all_results.append({
            "query": query,
            "results": unique_results
        })

# Summarize all_results
total_unique_docs = sum(len(entry['results']) for entry in all_results)
total_queries_with_results = len(all_results)
all_doc_ids = set()
for entry in all_results:
    for res in entry['results']:
        doc_id = res.get('doc_id') or res.get('summary_metadata', {}).get('doc_id')
        if doc_id:
            all_doc_ids.add(doc_id)

print(f"Total unique documents retrieved: {len(all_doc_ids)}")
print(f"Total queries with at least one unique result: {total_queries_with_results}")
print("Number of unique documents retrieved per query:")
for entry in all_results:
    print(f"  Query: {entry['query'][:60]}... -> {len(entry['results'])} unique docs")



# 